# Coeficientes de reflexión a incidencia normal para distintos 'targets'

## Coeficientes de reflexion y transmision de fresnell (genericos)

In [12]:
def R_ij(ep_i,ep_j,tita_i,tita_j):
    '''
    Coeficiente de reflexion de fresnell entre capa i y j
    '''
    Rij = (np.sqrt(ep_i)*np.cos(tita_i)-np.sqrt(ep_j)*np.cos(tita_j))/(np.sqrt(ep_i)*np.cos(tita_i)+np.sqrt(ep_j)*np.cos(tita_j))
    return Rij

In [13]:
def T_ij(ep_i,ep_j,tita_i,tita_j):
    '''
    Coeficiente de transmicion de fresnell entre capa i y j
    '''
    Tij = (2*np.sqrt(ep_i)*np.cos(tita_i))/(np.sqrt(ep_i)*np.cos(tita_i)+np.sqrt(ep_j)*np.cos(tita_j))
    return Tij

## Un dieléctrico

### Target semi-infinito con constante dielectrica $\epsilon$. incidencia normal

In [14]:
def R_1capa(ep):
    '''
    Coeficiente de reflexion de fresnell (capa seminf)
    obs: no depende de la frecuencia
  
    Entradas:
    -ep: Epsilon (cte dielectrica del medio)

    Salidas:
    -R
    '''
    r=(np.sqrt(ep)-1)/(np.sqrt(ep)+1)
    return r


### target de espesor $d$ y constante dieléctrica $\epsilon$ y conductor ideal debajo del dieléctrico.

In [1]:
def R_ep_cond(ep,d,f): # antes era Ref2. Solo valido para frecuencias positivas.
    '''
    coef dieléctrico ep de espesor d sobre conductor ideal
    '''
    
    ki = 2*np.pi*f/3e8  
    fi = 2*1j*np.sqrt(ep)*ki*d
    
    a = 1-np.sqrt(ep)+(1+np.sqrt(ep))*np.exp(fi)
    b = 1+np.sqrt(ep)+(1-np.sqrt(ep))*np.exp(fi)
    
    return a/b

### target de espesor $d$ y constante dieléctrica $\epsilon_1$ y conductor real con conductividad $\sigma(\omega)$ debajo del dieléctrico.

In [ ]:
def R_ep_cond_ideal(ep1,d,sigma,f):

  w = 2*np.pi*f
  k1 = np.sqrt(ep1)*w/3e8

  r01 = (np.sqrt(ep1)-1)/(np.sqrt(ep1)+1)
  #ep_c = 1 + 1j*4*np.pi*sigma/w 
  #r1c = (np.sqrt(ep_c)-np.sqrt(ep1))/(np.sqrt(ep_c)+np.sqrt(ep1))
  r1c = 1-np.sqrt((2*w)/(np.pi*sigma))

  fase = 2*1j*k1*d

  a = r01+r1c*np.exp(fase)
  b = 1+r01*r1c*np.exp(fase)
  r = a/b

  return r

## Dos dieléctricos

### $\epsilon_1$ con espesor $d$ y luego $\epsilon_2$ semi-infinito

In [17]:
def R_ep1_ep2(ep1,ep2,d,f):
    '''
    dos dieléctricos: ep1 de espesor d y ep2 semi-infinito.
    
    config: aire-ep1-ep2
    '''
    
    ki = 2*np.pi*f/3e8 
    fi = 2*1j*np.sqrt(ep1)*ki*d
    r01 = (np.sqrt(ep1)-1)/(np.sqrt(ep1)+1)
    r12 = (np.sqrt(ep2)-np.sqrt(ep1))/(np.sqrt(ep2)+np.sqrt(ep1))

    r = (r01+r12*np.exp(fi))/(1+r01*r12*np.exp(fi))
    
    return r
    

### target de espesor $d_1 + d_2 = d$, donde la constante dieléctrica varía en dos tramos: $\epsilon_1$ arriba, $\epsilon_2$ debajo, luego el conductor ideal. 

In [19]:
def R_2capas(ep1,ep2,d1,d2,f):
    '''
    dos dieléctricos: ep1 con espesor d1, ep2 con espesor d2;
    ep1 sobre ep2 y ep2 sobre conductor ideal.
    
    config: aire-ep1-ep2-conductor.
    '''
    
    k0 = 2*np.pi*f/3e8

    #codigo de mariano de labo7
    k1 = np.sqrt(ep1)*k0
    k2 = np.sqrt(ep2)*k0

    #ex-codigo tesis
    #k1 = np.sqrt(np.real(ep1))*k0
    #k2 = np.sqrt(np.real(ep2))*k0
    
    fi1 = 2*1j*k1*d1
    fi2 = 2*1j*k2*d2

    r01 = (np.sqrt(ep1)-1)/(np.sqrt(ep1)+1) # interface ep0-ep1; ep0 = 1
    r12 = (np.sqrt(ep2)-np.sqrt(ep1))/(np.sqrt(ep2)+np.sqrt(ep1))  # interface ep1-ep2, dentro del dieléctrico

    a = r01 + r12*np.exp(fi1) + r01*r12*np.exp(fi2) + 1*np.exp(fi1+fi2)
    b = 1 + r01*r12*np.exp(fi1) + r12*np.exp(fi2) + r01*np.exp(fi1+fi2)

    R = a/b

    return R

## Tres dieléctricos

### Coeficiente de fresnel para 3 capas

In [20]:
def R_3capas(ep1, ep2, ep3, d1, d2, f):
    '''
  Coeficiente de reflexion de fresnell 2 interfases
    Entradas:
    -ep1: ϵ de la 1ra capa
    -ep2: ϵ de la 2da capa
    -ep: ϵ de la 3ra capa
    -d1: ancho de la 1er capa
    -d2: ancho de la 2da capa
    -f: frecuencia incidente

    Salidas:
    -R
  '''
    phi1 = (2*np.pi*f*np.sqrt(ep1)*d1)/3e8
    phi2 = (2*np.pi*f*np.sqrt(ep2)*d2)/3e8
    R01 = (np.sqrt(ep1)-1)/(1+np.sqrt(ep1))
    R12 = (np.sqrt(ep2)-np.sqrt(ep1))/(np.sqrt(ep1)+np.sqrt(ep2))
    R23 = (np.sqrt(ep3)-np.sqrt(ep2))/(np.sqrt(ep2)+np.sqrt(ep3))
    
    RR1=(R12+R23*np.exp(2*1j*phi2))/(1+R12*R23*np.exp(2*1j*phi2))
    
    a = R01+RR1*np.exp(2*1j*phi1)
    b = 1+R01*RR1*np.exp(2*1j*phi1)
    
    R = a/b
    return R